# What is TrueFoundry❓❓❓
1. An ML Platforms - training jobs, model registry, experiment tracking, inference services, notebooks
2. Deployments - deploy code in minutes, multi-cloud, best practices out of the box
3. Software Catalogues - manage users, provisions resources, all your workloads under one place


#🤔 How does it work?
![image](https://i.imgur.com/MTr5wcZ.png)


#💻 What can you do with TrueFoundry?
1. Deploy a service from code on Github
2. Setup a cron job using Python code
3. Create an ML repo and store your artifacts
4. More Below 👇

In [ ]:
!pip install -q mlfoundry servicefoundry transformers datasets transformers[torch] gradio

# Let's get setup 🏋

In [ ]:
import os
import getpass
import logging
import random
import string

print(random.choices(string.ascii_lowercase))
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.WARN, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

os.environ["TFY_HOST"] = "https://carelon.truefoundry.cloud"
print(f"Set API Key (You can generate one at {os.getenv('TFY_HOST')}/settings/?tab=api-keys)")
os.environ["TFY_API_KEY"] = getpass.getpass(f"Enter API Key: ")

In [ ]:
REPO_NAME = input('give a unique name for your ML repository: ')

# Creating a ML repository

In [ ]:
import mlfoundry
mlf_client = mlfoundry.get_client()
repo = mlf_client.create_ml_repo(REPO_NAME)

# What is experiment tracking?
Quiz?
1. What are the different things we track during an experiment?
2. Why is it important?

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the iris dataset as an example
iris = datasets.load_iris()
X = iris.data
y = iris.target

# create an MLFoundry run
run = mlf_client.create_run(ml_repo=REPO_NAME, run_name='sklearn-run')
run.set_tags({
    'dataset': 'iris'
})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# store our hyperparams
run.log_params({
    'test_size': 0.7,
    'random_state': 42
})

# Create a logistic regression model
model = LogisticRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Log metrics
# store our hyperparams
run.log_metrics({
    'accuracy': accuracy
})

# Log model
model_version = run.log_model(
    name="my-sklearn-model",
    model=model,
    framework="sklearn"
)

# Tracking a TrueFoundry job
https://github.com/truefoundry/llm-training-notebooks/tree/main/jobs/iris-classification

# Now let's track an LLM training job

In [ ]:
import os
import random

import numpy as np
import pandas as pd

from IPython.display import display, HTML

import datasets
from datasets import load_dataset, load_metric
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# let's id the model and load datasets
TASK = "classification"
DATASET = "tweet_eval"
SUBSET = "emotion"
MODEL_CHECKPOINT = "google/bert_uncased_L-2_H-128_A-2"

dataset = load_dataset(DATASET, SUBSET)
metric = load_metric("accuracy")

In [ ]:
# sample data
dataset['train'][0]

In [ ]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, use_fast=True)
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

In [ ]:
sentence1_key, sentence2_key = "text", None
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True, max_length=256)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, max_length=256)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
num_labels = dataset['train'].features['label'].num_classes
labels = dataset['train'].features['label'].names
label2id = dict(zip(labels, range(len(labels))))
id2label = {v: k for k, v in label2id.items()}

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# load the model and config for training
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT, label2id=label2id, id2label=id2label)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, config=config)

In [ ]:
metric_name = "accuracy"
batch_size = 256
epochs = 20
freq = 20
model_name = MODEL_CHECKPOINT.split("/")[-1]

# trainer arguments
args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-{DATASET}-{SUBSET}-{TASK}",
    logging_strategy="steps",
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=freq,
    save_strategy="steps",
    save_steps=freq,
    save_total_limit=3,
    learning_rate=9e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    warmup_ratio=0.5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    report_to=[],
)

In [ ]:
from mlfoundry.integrations.transformers import MlFoundryTrainerCallback, LogModelStrategy

# this callback is used to log our metrics automatically
mlf_cb = MlFoundryTrainerCallback(
    ml_repo=REPO_NAME,
    run_name=f"{model_name}-finetuned-hf".replace("_", "-"),
    flatten_params=True,
    log_model_strategy=LogModelStrategy.BEST_ONLY
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[mlf_cb]
)
trainer.train()

In [ ]:
run_fqn = input('let us fetch the run fqn from the mlrepo: ')

In [ ]:
from mlfoundry.integrations.transformers import HF_MODEL_PATH

run = mlf_client.get_run_by_fqn(run_fqn)
downloaded = run.download_artifact_deprecated(HF_MODEL_PATH)

In [ ]:
config = AutoConfig.from_pretrained(downloaded)
model = AutoModelForSequenceClassification.from_pretrained(downloaded, config=config)
model = model.eval()

In [ ]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]
pipe("I am not feeling great")

In [ ]:
def predict(input):
  output = pipe(input)
  return {obj['label']: obj['score'] for obj in output[0]}


In [ ]:
import gradio as gr

gr.Interface(fn=predict, inputs="text", outputs="label").launch(share=True)